<a href="https://colab.research.google.com/github/Roudranil/competitions/blob/main/dphi/datasprint%2324/Data_Sprint_24_Tinder_Millennial_Match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# download bullshits
---




In [46]:
!pip install feature_engine

     |████████████████████████████████| 153kB 8.5MB/s 
     |████████████████████████████████| 9.5MB 10.7MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [88]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future

In [89]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 164.6MB 89kB/s 
  Created wheel for h2o: filename=h2o-3.32.0.4-py2.py3-none-any.whl size=164670979 sha256=10bd5872df3cc863581b3a3f6b4153ca0abf0c8ebd0df03f271f386b322f71f1
  Stored in directory: /root/.cache/pip/wheels/65/f4/0d/c9bb958d70c2e015c968cb91cbd7f1b486933056d422337d75
Successfully built h2o


# load packages


---






In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [78]:
import sklearn

In [91]:
import h2o
h2o.init()
from h2o.estimators import H2OTargetEncoderEstimator

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp9ro3a55w
  JVM stdout: /tmp/tmp9ro3a55w/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp9ro3a55w/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_unknownUser_f6xcqq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [2]:
train_data = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/Tinder_Millennial_Match/train_set_label.csv")
test_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/Tinder_Millennial_Match/test_set_label.csv')
submission = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/Tinder_Millennial_Match/sample_submission.csv')

# random stuff
---

In [12]:
test_data

,ID,Segment type,Segment Description,Answer,Count,Percentage
0,292974.2311,university,"University of California, Davis",Yes,0.230742,0.115371
1,292889.7996,web,"Meridian, Idaho",No,0.000000,0.000000
2,293724.0350,university,"Westport, CT",Yes,0.000000,0.000000
3,292884.0724,mobile,"Meridian, Idaho",No,1497.699982,0.388752
4,292882.0964,mobile,"Meridian, Idaho",No,1439.464988,0.373646
...,...,...,...,...,...,...
627,293085.4757,university,University of Miami,Yes,1.000000,0.743904
628,292890.5771,web,"Meridian, Idaho",No,0.000000,0.000000
629,292891.5922,gender,"Meridian, Idaho",No,1104.827490,0.411368
630,292889.9699,web,"Meridian, Idaho",No,0.000000,0.000000


In [23]:
train_data

,ID,Segment type,Segment Description,Answer,Count,Percentage,It became a relationship
0,292890.8970,web,"Meridian, Idaho",No,0.000000,0.000000,0
1,292887.9870,web,"Meridian, Idaho",No,0.000000,0.000000,0
2,292894.0656,gender,"Meridian, Idaho",No,499.173606,0.225255,0
3,292887.1180,web,"Meridian, Idaho",No,0.000000,0.000000,0
4,292893.6561,gender,"Meridian, Idaho",No,455.925963,0.211360,0
...,...,...,...,...,...,...,...
1891,292887.5496,web,"Meridian, Idaho",No,0.000000,0.000000,0
1892,292881.6932,mobile,"Meridian, Idaho",No,1203.190399,0.312360,0
1893,292900.8499,gender,"Meridian, Idaho",No,806.378820,0.488025,0
1894,292893.8600,gender,"Meridian, Idaho",No,1149.529381,0.488984,0


In [6]:
train_data['Segment type'].unique()

array(['web', 'gender', 'university', 'mobile'], dtype=object)

In [9]:
train_data['Segment Description'].value_counts()

Meridian, Idaho                       1421
Westport, CT                            14
University of Pennsylvania              12
University of Washington                10
University of Mississippi               10
                                      ... 
Centre College                           1
Penn State University                    1
Castleton State College                  1
University of Wisconsin-Madison          1
Washington University in St. Louis       1
Name: Segment Description, Length: 148, dtype: int64

In [15]:
train_data['It became a relationship'].value_counts()

0    1231
1     665
Name: It became a relationship, dtype: int64

In [10]:
train_data['Answer'].unique()

array(['No', 'Yes', "I don't use Tinder"], dtype=object)

In [11]:
train_data.groupby('Segment type')['Count'].describe()

,count,mean,std,min,25%,50%,75%,max
Segment type,,,,,,,,
gender,488.0,696.694593,251.192948,325.000000,499.441557,562.983279,927.345270,1172.0
mobile,483.0,1044.103188,312.427120,794.126865,798.593448,930.728003,1077.826444,1969.0
university,464.0,6.461382,13.570453,0.000000,0.000000,1.000000,4.925250,95.0
web,461.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [25]:
train_data[train_data['ID'] == 292898.6134]

,ID,Segment type,Segment Description,Answer,Count,Percentage,It became a relationship
1170,292898.6134,gender,"Meridian, Idaho",No,334.472011,0.198967,0
1819,292898.6134,gender,"Meridian, Idaho",No,946.164059,0.503309,0


In [24]:
train_data['ID'].value_counts()

292898.6134    2
292888.8583    2
292889.1977    2
292882.7654    2
292890.6779    2
              ..
292897.7541    1
292899.9143    1
292890.7376    1
292891.3274    1
292890.3132    1
Name: ID, Length: 1888, dtype: int64

In [14]:
train_data.Count.sum()

847286.8819436389

In [19]:
common = pd.merge(how='inner', on='ID', left=train_data, right=test_data, indicator=True)

In [20]:
common

,ID,Segment type_x,Segment Description_x,Answer_x,Count_x,Percentage_x,It became a relationship,Segment type_y,Segment Description_y,Answer_y,Count_y,Percentage_y,_merge
0,292900.5822,gender,"Meridian, Idaho",No,400.548871,0.244279,0,gender,"Meridian, Idaho",No,804.913545,0.480765,both
1,292888.0153,web,"Meridian, Idaho",No,0.000000,0.000000,0,web,"Meridian, Idaho",No,0.000000,0.000000,both


In [31]:
train_data.groupby('Answer')['It became a relationship'].value_counts()

Answer              It became a relationship
I don't use Tinder  1                             81
                    0                             80
No                  0                           1071
                    1                            498
Yes                 1                             86
                    0                             78
Name: It became a relationship, dtype: int64

In [32]:
train_data['Answer'].value_counts()

No                    1569
Yes                    164
I don't use Tinder     161
Name: Answer, dtype: int64

In [33]:
test_data['Answer'].value_counts()

No                    522
I don't use Tinder     56
Yes                    54
Name: Answer, dtype: int64

In [60]:
l1 = train_data['Segment Description'].unique()
l2 = test_data['Segment Description'].unique()

new_seg = [seg for seg in l2 if seg not in l1]

In [61]:
new_seg

['Minnesota State University Mankato',
 'Oregon State University',
 'Bucknell University']

# preprocessing
----

In [ ]:
# pipeline 1

train_data.drop_duplicates(inplace=True)

train = train_data.drop(columns='ID')
test = test_data.drop(columns="ID")

answer = {'No': 0, 'Yes': 1, "I don't use Tinder":0}
train['Answer'] = train['Answer'].map(answer)
test['Answer'] = test['Answer'].map(answer)

from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(drop='first', sparse=False)
encoded_train = pd.DataFrame(onehot.fit_transform(train.iloc[:, 0:1]), index=train.index)
encoded_test = pd.DataFrame(onehot.transform(test.iloc[:, 0:1]), index=test.index)
train = pd.concat([encoded_train, train.drop(columns='Segment type')], axis=1)
test = pd.concat([encoded_test, test.drop(columns='Segment type')], axis=1)

from feature_engine.encoding import CountFrequencyEncoder
cfe = CountFrequencyEncoder(encoding_method='frequency')
train['Segment Description'] = cfe.fit_transform(train.iloc[:, 3:4])
test['Segment Description'] = cfe.transform(test.iloc[:, 3:4])

from sklearn.impute import SimpleImputer
imp = SimpleImputer()
test['Segment Description'] = imp.fit_transform(test.iloc[:, 3:4])

In [117]:
# pipeline 2

train_data.drop_duplicates(inplace=True)

train1 = train_data.drop(columns='ID')
test1 = test_data.drop(columns="ID")

hf_tr = h2o.H2OFrame(train1)
hf_test = h2o.H2OFrame(test1)
fold_column = "kfold_column"
hf_tr[fold_column] = hf_tr.kfold_column(n_folds=5, seed=123)
enc = H2OTargetEncoderEstimator(data_leakage_handling="k_fold",
                                blending=True,
                                inflection_point=3,
                                smoothing=10,
                                noise=0.15,     # In general, the less data you have the more regularization you need
                                seed=123,
                                fold_column=fold_column)
enc.train(x='Answer', y='It became a relationship', training_frame=hf_tr)
hf_tr = enc.transform(hf_tr, as_training=True)
hf_test = enc.transform(hf_test, noise=0)

train1 = hf_tr.as_data_frame()
train1.drop(columns=['Answer', 'kfold_column'], inplace=True)
test1 = hf_test.as_data_frame()
test1.drop(columns=['Answer'], inplace=True)

# check iloc indexes

# from sklearn.preprocessing import OneHotEncoder
# onehot = OneHotEncoder(drop='first', sparse=False)
# encoded_train1 = pd.DataFrame(onehot.fit_transform(train.iloc[:, 0:1]), index=train.index)
# encoded_test1 = pd.DataFrame(onehot.transform(test.iloc[:, 0:1]), index=test.index)
# train = pd.concat([encoded_train, train.drop(columns='Segment type')], axis=1)
# test = pd.concat([encoded_test, test.drop(columns='Segment type')], axis=1)

# from feature_engine.encoding import CountFrequencyEncoder
# cfe = CountFrequencyEncoder(encoding_method='frequency')
# train['Segment Description'] = cfe.fit_transform(train.iloc[:, 3:4])
# test['Segment Description'] = cfe.transform(test.iloc[:, 3:4])

# from sklearn.impute import SimpleImputer
# imp = SimpleImputer()
# test['Segment Description'] = imp.fit_transform(test.iloc[:, 3:4])

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
targetencoder Model Build progress: |█████████████████████████████████████| 100%


# Model
---

In [75]:
y = train['It became a relationship']
X = train.drop(columns='It became a relationship')

In [72]:
from sklearn.model_selection import KFold

In [80]:
cv = KFold(n_splits=4, shuffle=True, random_state=123)

In [74]:
from sklearn.metrics import accuracy_score as acc

In [81]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
xgbc = XGBClassifier()

scores = cross_val_score(xgbc, X, y, cv=cv, scoring='accuracy')

In [82]:
scores.mean()

0.9197531690172256

# submission
---

In [84]:
xgbc.fit(X, y)
submission['prediction'] = xgbc.predict(test)

In [85]:
submission.iloc[444, 0] = 0

In [86]:
submission.to_csv('submission2.csv', index=False)